In [1]:
def start_end_decorator(func):
    
    def wrapper():
        print('Start')
        func()
        print('End')
    return wrapper

def printname():
    print('Elsa')

printname = start_end_decorator(printname) 
printname()

Start
Elsa
End


Now the printname function has this new functionality. 
It prints 'Start', then it executes the function and prints 'Elsa' and then it prints 'End'.

Now the decorator function will do the same thing as the line: "  printname = start_end_decorator(printname)  "

In [2]:
def start_end_decorator(func):
    
    def wrapper():
        print('Start')
        func()
        print('End')
    return wrapper

@start_end_decorator
def printname():
    print('Elsa')
    
printname()

Start
Elsa
End


Let's say we have a function that takes an argument.

In [4]:
def start_end_decorator(func):
    
    def wrapper(*args, **kwargs):  # We this syntax I can use as many arguments and keywords arguments as I want
        print('Start')
        func(*args, **kwargs)  
        print('End')
    return wrapper

@start_end_decorator
def add5(x):
    return 5

add5(10)

Start
End


It works! But what about the return value?

In [8]:
result = add5(10)
print(result)

Start
End
None


It prints None. To fix this, I need to save the result and return it in the wrapper.

In [9]:
def start_end_decorator(func):
    
    def wrapper(*args, **kwargs):
        print('Start')
        result = func(*args, **kwargs)  
        print('End')
        return result
    return wrapper

@start_end_decorator
def add5(x):
    return 5

result = add5(10)
print(result)

Start
End
5


In [10]:
print(help(add5))
print(add5.__name__)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)

None
wrapper


Python got confused about the identity of add5 function. In order to fix this, I can import functools and apply a decorator before wrapper.

In [11]:
import functools

In [13]:
def start_end_decorator(func):
    #this will now preserve the information of my used function
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('Start')
        result = func(*args, **kwargs)  
        print('End')
        return result
    return wrapper

@start_end_decorator
def add5(x):
    return 5

In [14]:
print(help(add5))
print(add5.__name__)

Help on function add5 in module __main__:

add5(x)
    #this will now preserve the information of my used function

None
add5


In [15]:
def repeat(num_times):
    
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            for _ in  range(num_times):  #I put "_" because I don't need it
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator_repeat

@repeat(num_times=3) #executes this function 3 times
def greet(name):
    print(f'Hello {name}')
    
greet('Elsa')

Hello Elsa
Hello Elsa
Hello Elsa


Let's talk about nested decorators. You can stack decorators on top of each other.

In [17]:
def debug(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        args_repr = [repr(a) for a in args]  #extracts the name in the arguments
        kwargs_repr = [f"{k}={v!r}" for k,v in kwargs.items()]  #and the keyword arguments
        signature = ", ".join(args_repr + kwargs_repr)
        print(f"Calling {func.__name__}({signature})")  #prints the information of this function
        result = func(*args, **kwargs)  #it executes the function
        print(f"{func.__name__!r} returned {result!r}")  #prints the information about the return value
        return result
    return wrapper
    

#the decoraters will be executed in the order they are listed
@debug
@start_end_decorator
def say_hello(name):
    greeting = f'Hello {name}'
    print(greeting)
    return greeting

#1. Executes the debug function
#2. Inside the debug function, , it executes the start_end_decorator function
#3. Inside the start_end_decorator function, it executes the say_hello function
say_hello('Elsa')

Calling say_hello('Elsa')
Start
Hello Elsa
End
'say_hello' returned 'Hello Elsa'


'Hello Elsa'

Let's talk about class decorators. Class decorators do the same thing as function decorators but thay are typically used if we want to maintain and update a state.

In this examples, I want to keep track of how many times I have executed a function.

In [19]:
class CountCalls:
    
    def __init__(self,func):
        self.func = func
        self.num_calls = 0
        
    def __call__(self, *args, **kwargs):
        print('Hi there')
        
cc = CountCalls(None)
cc()

@CountCalls
def say_hello():
    print('Hello')

Hi there


I don't want to print 'Hi there'. I want to update the state.

In [21]:
class CountCalls:
    
    def __init__(self,func):
        self.func = func
        self.num_calls = 0
        
    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f'This is executed {self.num_calls} times')
        return self.func(*args, **kwargs)
        

@CountCalls
def say_hello():
    print('Hello')

say_hello()

This is executed 1 times
Hello


In [22]:
say_hello()

This is executed 2 times
Hello
